In [1]:
import os
import glob
from tqdm import tqdm 
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import random
from torch.autograd import Variable
import cv2
from torchvision import transforms
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [3]:
class DogsVSCats():
    IMG_SIZE = 50
    CATS = "C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/cats/"
    DOGS="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/dogs/"
    TESTING = "C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

NameError: name 'REBUILD_DATA' is not defined

In [4]:
IMG_SIZE = 50
CATS = "C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/cats/"
DOGS="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/dogs/"
TESTING = "C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set"
LABELS = {CATS: 0, DOGS: 1}
training_data = []
catcount = 0
dogcount = 0

for label in LABELS:
    print(label)
    for f in tqdm(os.listdir(label)):
        if "jpg" in f:
            path = os.path.join(label, f)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img), np.eye(2)[LABELS[label]]])
            
            if label == CATS:
                catcount += 1
            elif label == DOGS:
                dogcount += 1
    
    
np.random.shuffle(training_data)
np.save("training_data.npy",training_data)
print('Cats:',catcount)
print('Dogs:',dogcount)

C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/cats/


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:02<00:00, 15.99it/s]


C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/dog vs cat/dataset/test_set/dogs/


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 22.09it/s]


Cats: 1000
Dogs: 1000


In [5]:
len(training_data)

2000

In [6]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [8]:
X.shape,y.shape

(torch.Size([2000, 50, 50]), torch.Size([2000, 2]))

In [9]:
VAL_PCT = 1 # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

2000


In [10]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [43]:
len(train_X),len(train_y),len(test_X),len(test_y)

(0, 0, 2000, 2000)

In [11]:
train_X.shape,train_y.shape,test_X.shape,test_y.shape

(torch.Size([0, 50, 50]),
 torch.Size([0, 2]),
 torch.Size([2000, 50, 50]),
 torch.Size([2000, 2]))

In [15]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [16]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss = nn.MSELoss()

In [34]:
BATCH_SIZE = 100
EPOCHS = 10

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss.data}")

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [41]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(100)):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
        print("Accuracy: ", round(correct/total, 3))

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Accuracy:  0.0
Accuracy:  0.5
Accuracy:  0.667
Accuracy:  0.5
Accuracy:  0.4
Accuracy:  0.333


  6%|████▉                                                                             | 6/100 [00:00<00:01, 49.72it/s]

Accuracy:  0.286
Accuracy:  0.25
Accuracy:  0.333
Accuracy:  0.4


 10%|████████                                                                         | 10/100 [00:00<00:02, 44.53it/s]

...

 93%|███████████████████████████████████████████████████████████████████████████▎     | 93/100 [00:02<00:00, 44.57it/s]

Accuracy:  0.404
Accuracy:  0.411
Accuracy:  0.406
Accuracy:  0.402
Accuracy:  0.398
Accuracy:  0.394


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:02<00:00, 47.86it/s]

Accuracy:  0.4


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 38.79it/s]


In [36]:
EPOCHS=10
for epoch in range(EPOCHS):
    for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        image= train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        label = train_y[i:i+BATCH_SIZE]
        
        optimizer.zero_grad()
        
        output=model(image)
        
        loss=criterion(output,label)
        
        loss.backward()
        
        optimizer.step()
        
        iter+=1
        if iter%500==0:
            correct=0
            
            total=0
            
            for image,label in test_loader:
            
                image=Variable(image)
            
                output=model(image)
            
                _,prediction=torch.max(output.data,1)
            
                total+=label.size(0)
            
                correct+= (prediction == label).sum()
            
                accuracy=100*correct/total
        
            print("Iteraion : {} , Loss : {} , Accuracy : {}".format(iter,loss.data,accuracy))

In [7]:
import glob

In [59]:
data="./dog vs cat/dataset/test_set/"

In [46]:
transfrom_data=transforms.Compose([transforms.Compose((32,32)),transforms.ToTensor(),
                                  transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [60]:
dog=glob.glob(data+"dogs/*.jpg")
cat=glob.glob(data+"cats/*.jpg")

In [61]:
import torchvision

In [62]:
l=dog+cat

In [63]:
len(l)

2000

In [64]:
img_dataset=torchvision.datasets.ImageFolder(root=data,transform=transfrom_data)

In [65]:
train_set,test_set=torch.utils.data.random_split(img_dataset,(2000,600))

ValueError: Sum of input lengths does not equal the length of the input dataset!

In [18]:
6000/1553

3.86349001931745

In [68]:
2000/590

3.389830508474576